# Chapter 7 Applied Exercises

## Problem 6

### In this exercise, you will further analyze the Wage data set considered throughout this chapter.

#### Part (a)

Perform polynomial regression to predict wage using age. Use cross-validation to select the optimal degree d for the polynomial. What degree was chosen, and how does this compare to
the results of hypothesis testing using ANOVA? Make a plot of the resulting polynomial fit to the data.

#### Part (b)

Fit a step function to predict wage using age, and perform cross-validation to choose the optimal number of cuts. Make a plot of the fit obtained.

## Problem 7

The Wage data set contains a number of other features not explored in this chapter, such as marital status (maritl), job class (jobclass), and others. Explore the relationships between some of these other predictors and wage, and use non-linear fitting techniques in order to fit flexible models to the data. Create plots of the results obtained, and write a summary of your findings.

## Problem 8

Fit some of the non-linear models investigated in this chapter to the Auto data set. Is there evidence for non-linear relationships in this data set? Create some informative plots to justify your answer.

## Problem 9

This question uses the variables dis (the weighted mean of distances to five Boston employment centers) andnox (nitrogen oxides concentration in parts per 10 million) from the Boston data. We will treat dis as the predictor and nox as the response.

#### Part (a)

Use the poly() function from the ISLP.models module to fit a cubic polynomial regression to predict nox using dis. Report the regression output, and plot the resulting data and polynomial fits.

#### Part (b)

Plot the polynomial fits for a range of different polynomial degrees (say, from 1 to 10), and report the associated residual sum of squares.

#### Part (c)

Perform cross-validation or another approach to select the optimal degree for the polynomial, and explain your results.

#### Part (d)

Use the bs() function from the ISLP.models module to fit a regression spline to predict nox using dis. Report the output for the fit using four degrees of freedom. How did you choose the knots? Plot the resulting fit.

#### Part (e)

Now fit a regression spline for a range of degrees of freedom, and plot the resulting fits and report the resulting RSS. Describe the results obtained.

#### Part (f)

Perform cross-validation or another approach in order to select the best degrees of freedom for a regression spline on this data. Describe your results.

## Problem 10

This question relates to the College data set.

#### Part (a)

Split the data into a training set and a test set. Using out-of-state tuition as the response and the other variables as the predictors, perform forward stepwise selection on the training set in order to identify a satisfactory model that uses just a subset of the predictors.

#### Part (b)

Fit a GAM on the training data, using out-of-state tuition as the response and the features selected in the previous step as the predictors. Plot the results, and explain your findings.

#### Part (c)

Evaluate the model obtained on the test set, and explain the results obtained.

#### Part (d)

For which variables, if any, is there evidence of a non-linear relationship with the response?

## Problem 11

### In Section 7.7, it was mentioned that GAMs are generally fit using a backfitting approach. 
### The idea behind backfitting is actually quite simple. We will now explore backfitting in the context of multiple linear regression.

Suppose that we would like to perform multiple linear regression, but we do not have software to do so. Instead, we only have software to perform simple linear regression. Therefore, we take the following iterative approach: we repeatedly hold all but one coefficient estimate fixed at its current value, and update only that coefficient estimate using a simple linear regression. The process is continued until convergence—that is, until the coefficient estimates stop changing.

We now try this out on a toy example.

#### Part (a)

Generate a response $Y$ and two predictors $X_1$ and $X_2$, with
$n = 100$.

#### Part (b)

Write a function simple_reg() that takes two arguments outcome and feature, fits a simple linear regression model with this outcome and feature, and returns the estimated intercept and slope.

#### Part (c)

Initialize beta1 to take on a value of your choice. It does not
matter what value you choose.

#### Part (d)

Keeping beta1 fixed, use your function simple_reg() to fit the model:
$$Y− beta1· X_1 = β_0 + β_2X_2 + ϵ.$$
Store the resulting values as beta0 and beta2.

#### Part (e)

Keeping beta2 fixed, fit the model
$$Y− beta2· X_2 = β_0 + β_1X_1 + ϵ.$$
Store the result as beta0 and beta1 (overwriting their previous
values).

#### Part(f)

Write a for loop to repeat (c) and (d) 1,000 times. Report the
estimates of beta0, beta1, and beta2 at each iteration of the for
loop. Create a plot in which each of these values is displayed,
with beta0, beta1, and beta2.

#### Part (g)

Compare your answer in (e) to the results of simply performing multiple linear regression to predict $Y$ using $X_1$ and $X_2$.
Use axline() method to overlay those multiple linear regression coefficient estimates on the plot obtained in (e).

#### Part (h)

On this data set, how many backfitting iterations were required in order to obtain a “good” approximation to the multiple regression coefficient estimates?

## Problem 12

This problem is a continuation of the previous exercise. In a toy
example with p = 100, show that one can approximate the multiple
linear regression coefficient estimates by repeatedly performing simple
linear regression in a backfitting procedure. How many backfitting
iterations are required in order to obtain a “good” approximation to
the multiple regression coefficient estimates? Create a plot to justify
your answer.